# Text Chunking for Noun Phrase

In [1]:
print(__doc__)  # Attribute prints '''remarks'''
'''These are remarks about text Chunking of Noun Phrases'''

Automatically created module for IPython interactive environment


'These are remarks about text Chunking of Noun Phrases'

In [2]:
# Needed for Noun Phrase work-around
#from io import StringIO  # from StringIO import cStringIO  #Python2
import os
import time
import sys
import json

from nltk.tokenize import sent_tokenize
import nltk.chunk
from nltk.tag import pos_tag
#from nltk.draw.tree import TreeSegmentWidget, tree_to_treesegment

import itertools
import collections
import re

import numpy as np
import pandas as pd


In [3]:
## Read 'PublicRemarks' Text from file (00m:31s)
textfile = open('PublicRemarksCorpus.txt', 'r') # File
text = textfile.read()  # String
#text = text[0:493]

## Convert Corpus to list of sentences (00m:26s)
df = pd.Series(sent_tokenize(text))
print('Number of sentences:',len(df))
#df = df[0:400]
df[0:4]

Number of sentences: 399276


0    This property OUTSTANDING offers outdoor livin...
1      Enjoy your private resort in your own backyard.
2                           Indoors is also remodeled.
3                        Kitchen opens to Family Room.
dtype: object

In [4]:
## Convert Corpus to list of sentences (00m:26s)
df = pd.Series(sent_tokenize(text))
print('Number of sentences:',len(df))
#df = df[0:400]
df[0:4]

Number of sentences: 399276


0    This property OUTSTANDING offers outdoor livin...
1      Enjoy your private resort in your own backyard.
2                           Indoors is also remodeled.
3                        Kitchen opens to Family Room.
dtype: object

In [5]:
## Clean Text (00m:07s)
df = df.apply(lambda x : re.sub(r'[\b\(\)\\\"\'\/\[\]\s+\,\.:\?!;#*=+]', ' ', x))  # '-' is under debate
df = df.apply(lambda x : re.sub( '\s+', ' ', x).strip() )  # remove white space

# Test
print('df has length: %i\n' % len(df))
print(df[0:4])

df has length: 399276

0    This property OUTSTANDING offers outdoor livin...
1       Enjoy your private resort in your own backyard
2                            Indoors is also remodeled
3                         Kitchen opens to Family Room
dtype: object


In [ ]:
## Gather noun phrase (NP) from sentence tree (01h:24m:--s)

start_time = time.time()

    # Define several tag patterns
grammar = r"""
      NP: {<DT|PP\$>?<JJ>*<NN>}   # chunk determiner/possessive, adjectives and noun
          {<NNP>+}                # chunk sequences of proper nouns
          {<NN>+}                 # chunk consecutive nouns
          {<CD>+}                 # chunk cardinal numbers i.e. Zip Codes
          {<JJ>+}                 # chunk hyphenated terms
          """
def extract_np(parsed_sent):
      for subtree in parsed_sent.subtrees():
        if subtree.label() == 'NP':
          yield ' '.join(word for word, tag in subtree.leaves())

def nounPhrase(sentence, grammar):
    x=[]  # declare x as list
    if len(sentence) < 1: 
        pass
    else:
        tagged_sent = pos_tag(sentence.split())  # List of tuples with [(Word, PartOfSpeech)]
        cp = nltk.RegexpParser(grammar)
        parsed_sent = cp.parse(tagged_sent)
        for npstr in extract_np(parsed_sent):
            x.append(npstr)  #print(npstr)
    return x

df2 = df.apply(lambda x : nounPhrase(x, grammar))

stop_time = time.time()

print("Runtime: %.2fs" % (stop_time - start_time))
os.system("say 'Computation Complete!'")

df2[0:10]

In [ ]:
## All Noun Phrases
NounPhrases = list(np.concatenate(df2))  # chain = np.concatenate(df2), then convert type to list

# Test
#print(NounPhrases[0:10])
#print(len(NounPhrases)) # 2.0 M

## NounPhrases - Save and Load

In [ ]:
## Save NounPhrases to JSON
## ====> DANGER <==== Don't overwrite your NounPhrases!!!
#with open('NounPhrases.json', 'w') as outfile:
#    json.dump(NounPhrases, outfile)

In [ ]:
## Read Dict from file
f = open('NounPhrases.json','r')
data = f.read()
NPs = json.loads(data)

##Test
print(NPs[0:10])
print(len(NPs))
print(type(NPs))

## Conversion to n-grams

In [ ]:
## NounPhraseList (order & count are lost! Sorted instead.)
NounPhraseList = sorted(list(set(map(lambda x:x.lower(),NPs))))  ## Create 'NPList', a set of NounPhrases of type list

#Test
print(NounPhraseList[17000:17020])
print(len(NounPhraseList))

In [ ]:
## Save n-grams to JSON
## ====> DANGER <==== Don't overwrite your ngrams!!!
#with open('ngrams.json', 'w') as outfile:
#    json.dump(ngrams, outfile)

In [ ]:
## Create Counter
# Ex. a = [1,1,1,1,2,2,2,2,3,3,4,5,5]
#counter = collections.Counter(dictionary)  # Specialized Dictionary
#print(counter)
# Ex. Counter({1: 4, 2: 4, 3: 2, 5: 2, 4: 1})
#print(counter.values())
# Ex. [4, 4, 2, 1, 2]
#print(counter.keys())
# Ex. [1, 2, 3, 4, 5]
#print(counter.most_common(500))

#grams = counter.most_common(20)  # top unigrams
#grams[0:10]

## Thoughts
# probably worth comparing to the nltk n-gram tool

In [ ]:
## Example of Boolean Variance calculation
from sklearn.feature_selection import VarianceThreshold
#X = [[0, 0, 1], [0, 1, 0], [1, 0, 0], [0, 1, 1], [0, 1, 0], [0, 1, 1]]
P_True = 0.8
sel = VarianceThreshold(threshold=(P_True * (1 - P_True)))  # Boolean Var() clac
sel.fit_transform(X)

In [ ]:
## Univariate Feature Selection
## Source: http://scikit-learn.org/stable/modules/feature_selection.html
from sklearn.datasets import load_iris
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2
iris = load_iris()
X, y = iris.data, iris.target
print(X.shape)
X_new = SelectKBest(chi2, k=2).fit_transform(X, y)
print(X_new.shape)
print(type(X_new))
print(X[0:10])
print(X_new[0:10])

In [ ]:
# Read Dictionary from file
f = open('dictionary.json','r')
data = f.read()
dictionary = json.loads(data)
dictionary[0:10]

In [ ]:
## New approach

In [ ]:
# Read Dictionary from file
f = open('PublicRemarksCorpus.txt','r')
corpus = f.read()
corpus[0:100]

In [ ]:
def extract_candidate_words(text, good_tags=set(['JJ','JJR','JJS','NN','NNP','NNS','NNPS'])):
    import itertools, nltk, string

    # exclude candidates that are stop words or entirely punctuation
    punct = set(string.punctuation)
    stop_words = set(nltk.corpus.stopwords.words('english'))
    # tokenize and POS-tag words
    tagged_words = itertools.chain.from_iterable(nltk.pos_tag_sents(nltk.word_tokenize(sent)
                                                                    for sent in nltk.sent_tokenize(text)))
    # filter on certain POS tags and lowercase all words
    candidates = [word.lower() for word, tag in tagged_words
                  if tag in good_tags and word.lower() not in stop_words
                  and not all(char in punct for char in word)]

    return candidates

In [ ]:
candidates = extract_candidate_words(corpus)
candidates[0:5]

In [ ]:
def score_keyphrases_by_tfidf(texts, candidates='chunks'):
    import gensim, nltk
    
    # extract candidates from each text in texts, either chunks or words
    if candidates == 'chunks':
        boc_texts = [extract_candidate_chunks(text) for text in texts]
    elif candidates == 'words':
        boc_texts = [extract_candidate_words(text) for text in texts]
    # make gensim dictionary and corpus
    dictionary = gensim.corpora.Dictionary(boc_texts)
    corpus = [dictionary.doc2bow(boc_text) for boc_text in boc_texts]
    # transform corpus with tf*idf model
    tfidf = gensim.models.TfidfModel(corpus)
    corpus_tfidf = tfidf[corpus]
    
    return corpus_tfidf, dictionary

In [ ]:
score_keyphrases_by_tfidf(corpus)

## Noun Phrase Importance using TextRank

In [1]:
import json

# Read Dictionary from file
f = open('NounPhrases.json','r')
NounPhrases = json.load(f)

# LowerCase
NounPhrases = [item.lower() for item in NounPhrases]

In [2]:
import networkx as nx
import numpy as np

#from nltk.tokenize.punkt import PunktSentenceTokenizer
from sklearn.feature_extraction.text import TfidfTransformer, CountVectorizer
 
def textrank(NounPhrases):
    #sentence_tokenizer = PunktSentenceTokenizer()
    #sentences = sentence_tokenizer.tokenize(document)
 
    bow_matrix = CountVectorizer().fit_transform(NounPhrases)
    normalized = TfidfTransformer().fit_transform(bow_matrix)
 
    similarity_graph = normalized * normalized.T
 
    nx_graph = nx.from_scipy_sparse_matrix(similarity_graph)
    scores = nx.pagerank(nx_graph)
    return sorted(((scores[i],s) for i,s in enumerate(NounPhrases)),
                  reverse=True)  # Output ordered rank of NounPhrases

In [3]:
ranked = textrank(NounPhrases[0:10000])  # [0:1000]

In [4]:
ranked[50]  # [0:5][1]

(0.0002575679147337191,
 'bath in great gilbert location all appliances are included tile in kitchen and bathrooms ceiling fans rv gate')

In [8]:
NounPhrases[0:9]

['this property',
 'outstanding',
 'outdoor',
 'living',
 'entertainment',
 '85254',
 'enjoy',
 'private resort',
 'own backyard']

## Document Summarization Using TextRank

This will identify the most relevant sentences.  
Source: https://joshbohde.com/blog/document-summarization

In [9]:
# Read Dictionary from file
f = open('PublicRemarksCorpus.txt','r')
corpus = f.read()
corpus[0:1028]

"This property OUTSTANDING offers outdoor living spaces, enabling you to have unrivaled entertainment in the famed 85254.  Enjoy your private resort in your own backyard. Indoors is also remodeled.  Kitchen opens to Family Room. Large glass doors lead to backyard oasis. Pool & outdoor living room/ramada are some of the many fantastic qualities. Spacious 4 bedrooms + spacious flex room or office.  5 Year Roof Warranty, some new windows too!  Already great home with unlimited possibilities. With 8/10 of a lush acre and lots of room to roam or add on. Amazing opportunity to own in the coveted 85254 zip code. PERFECT LOCATION. 10 minutes from 7 golf courses and 12 minutes from Sky Harbor airport.  See for yourself...you will LOVE Hurry before It's gone! Very special home in popular Arabian Views feels like you are on vacation!  Open, airy and neutral decor.  Travertine stone flooring. Split floor plan, 4 beds, 2 baths with updating in all the right places! Granite counters in kitchen and b

In [10]:
import networkx as nx
import numpy as np
 
from nltk.tokenize.punkt import PunktSentenceTokenizer
from sklearn.feature_extraction.text import TfidfTransformer, CountVectorizer
 
def textrank(document):
    sentence_tokenizer = PunktSentenceTokenizer()
    sentences = sentence_tokenizer.tokenize(document)
 
    bow_matrix = CountVectorizer().fit_transform(sentences)
    normalized = TfidfTransformer().fit_transform(bow_matrix)
 
    similarity_graph = normalized * normalized.T
 
    nx_graph = nx.from_scipy_sparse_matrix(similarity_graph)
    scores = nx.pagerank(nx_graph)
    return sorted(((scores[i],s) for i,s in enumerate(sentences)),
                  reverse=True)

In [11]:
ranked = textrank(corpus[0:102800])  # Up to 10k, before slow-down

In [12]:
ranked[0][1]

'Gorgeous single level home in Pinnacle Reserve.Enter thru the front door to a large open great room with lots of windows and natural light and views of the patio and pebbletec pool.There is a formal dining area off the great room that can also be used as an office space.Large eat-in kitchen with tons of cabinet and counter space overlooks family room.Split floorplan with a huge bonus room/play room/mother-in-law suite.Private master bedroom with lots of natural light,double sink vanity, separate shower/tub and walk-in closet.2nd and 3rd bedrooms are good size with accompanying full bath with double sinks.Interior of the house was recently repainted as well as the garage.Covered patio overlooks sparkling pool with South facing backyard with plenty of sunshine.'

In [14]:
ranked[1][1]

"North-South exposure.Buyer verify all info ***BEST PRICE ON SINGLE-FAMILY HOME IN TROON*** HOUSE SITS ON CHOICE LOT ON CUL-DE-SAC STREET * ENJOY THE AMAZING MOUNTAIN & DESERT VIEWS AS YOU RELAX IN ARIZONA-STYLE ON YOUR LARGE COVERED PATIO WITH BEAUTIFUL EXTERIOR LIGHTING & HEATED POOL & SPA * GREAT ATMOSPHERE FOR ENTERTAINING * SPACIOUS, FLOWING FLOOR PLAN * SPLIT MASTER BEDROOM WITH SEPARATE EXIT TO BACKYARD * MASTER BATH FEATURES DUAL-SINKS, SOAKING TUB & SEPARATE SHOWER PLUS WALK-IN CLOSET * 2 ADD'L BEDS & BATHS * FORMAL DINING & LIVING AREAS PLUS FAMILY ROOM WITH COZY GAS FIREPLACE * ISLAND-KITCHEN IS OPEN AND HAS LOTS OF CABINETS & COUNTER SPACE & BUILT-IN DESK * 2 & 1/2 CAR GARAGE WITH BUILT-IN CABINETS & EXTRA STORAGE SPACE * A GREAT HOME FOR FULL OR PART-TIME RESIDENTS * CLOSE TO GOLF, SHOPPING & RESTAURANTS!"

## Junk!

In [ ]:
print(type(SentenceTree))
print(SentenceTree)
print(NounPhrases)


In [ ]:
for subtree in chunked_sent.subtrees():
    if subtree.label() == 'NP': print(subtree)

In [ ]:
SentenceTree.draw()



In [ ]:
def traverseTree(tree):
    #print("tree:", tree)
    set = []
    for subtree in tree:
        if type(subtree) == nltk.tree.Tree:
            traverseTree(subtree)
        else:
            if tree.label() == 'NP':
                set += tree[0][0]  #print(tree[0][0])
    return set

cp = nltk.RegexpParser(grammar)
SentenceTree = cp.parse(tagged_sent)
#print(SentenceTree)

NounPhrases2 = traverseTree(SentenceTree)
print(NounPhrases2)

#SentenceTree.leaves()




In [ ]:
%matplotlib inline

#import urllib
#from bs4 import BeautifulSoup
import nltk
#from nltk.tokenize import word_tokenize
#from nltk.tokenize import RegexpTokenizer
#from nltk.collocations import *
#import string

In [ ]:
# Python code for segmentation, POS tagging and tokenization
rawtext = open('PublicRemarksCorpus.txt').read()
sentences = nltk.sent_tokenize(rawtext) # NLTK default sentence segmenter 
sentences = [nltk.word_tokenize(sent) for sent in sentences] # NLTK word tokenizer
sentences = [nltk.pos_tag(sent) for sent in sentences] # NLTK POS tagger

In [ ]:
Patterns = """
        NP:     {<DT|PP\$>?<JJ>*<NN>}
                {<NNP>+}
                {<NN>+}
"""

NPChunker = nltk.RegexpParser(patterns) 

### Create a corpus and conduct analysis

In [ ]:
## Concatenate 'PublicRemarks' into a corpus
def concatCol(col):
    text = ''  # Declare text as string
    for n in range(len(col)): # 
        text += str(col[n])+' '
    return text

corpus = concatCol(df['PublicRemarks'])

## Save Text to File
file = open('PublicRemarksCorpus.txt', "w")
file.write(corpus)
file.close()

In [ ]:
# Read 'PublicRemarks' Text from file
textfile = open('PublicRemarksCorpus.txt', 'r') # File
corpus = textfile.read()  # String
#corpus[0:500]

In [ ]:
import re
from nltk.tokenize import word_tokenize
from nltk.util import ngrams

def get_ngrams(text, n ):
    text = re.sub(r'[\b\(\)\\\"\'\/\[\]\s+\,\.:\?;]', ' ', text)
    text = text.lower()
    tokens = word_tokenize(text)
    n_grams = ngrams(tokens, n)
    return [ ' '.join(grams) for grams in n_grams]

unigrams = get_ngrams(corpus, 1)